# Kruskal Tensors

Kruskal format is a decomposition of a tensor $\mathcal{X}$ as the sum of the outer products as the columns of matrices. For example, we might write:

$
\mathcal{X} = \sum_{r} a_r \circ b_r \circ c_r
$

where a subscript denotes column index and a circle denotes outer product. In other words, the tensor X is built from the columns of the matrices A,B, and C. It's often helpful to explicitly specify a weight for each outer product, which we do here:

$
\mathcal{X} = \sum_{r} \lambda_r \, a_r \circ b_r \circ c_r
$

The **`ktensor`** class stores the components of the tensor $\mathcal{X}$ and can perform many operations, e.g., **`ttm`**, without explicitly forming the tensor $\mathcal{X}$.

## Kruskal tensor format via ktensor

In [ ]:
import pyttb as ttb
import numpy as np

Kruskal format stores a tensor as a sum of rank-1 outer products. For example, consider a tensor of the following form:

$
X = a_1 \circ b_1 \circ c_1 + a_2 \circ b_2 \circ c_2
$

This can be stored in Kruskal form as follows.

In [ ]:
np.random.seed(0)
A = np.random.rand(4,2) # First column is a_1, second is a_2.
B = np.random.rand(3,2) # Likewise for B.
C = np.random.rand(2,2) # Likewise for C.
X = ttb.ktensor.from_factor_matrices([A,B,C]) # Create the ktensor.
X

For Kruskal format, there can be any number of matrices, but every matrix must have the same number of columns. The number of rows can vary.

In [ ]:
Y = ttb.ktensor.from_factor_matrices(
    [np.random.rand(4,1), np.random.rand(2,1), np.random.rand(3,1)]
) # Another ktensor.
Y

## Specifying weights in a ktensor

Weights for each rank-1 tensor can be specified by passing in a column vector. For example: 

$
X = \lambda_1 \, a_1 \circ b_1 \circ c_1 + \lambda_2 \, a_2 \circ b_2 \circ c_2 $

In [ ]:
# Upcoming ktensors will be generated with this same initialization.
def generate_sample_ktensor()-> ttb.ktensor:
    np.random.seed(0)
    A = np.random.rand(4,2) # Create some data.
    B = np.random.rand(3,2)
    C = np.random.rand(2,2)
    weights = np.array([5.0, 0.25])
    
    return ttb.ktensor.from_data(weights, [A,B,C]) # Create the ktensor.

In [ ]:
X = generate_sample_ktensor()
X

## Creating a one-dimensional ktensor

In [ ]:
np.random.seed(0)
Y = ttb.ktensor.from_factor_matrices(np.random.rand(4,5)) # A one-dimensional ktensor.
Y

## Constituent parts of a ktensor

In [ ]:
X = generate_sample_ktensor()
X.weights # Weights or multipliers.

In [ ]:
X.factor_matrices # Cell array of matrices.

## Creating a ktensor from its constituent parts

In [ ]:
X = generate_sample_ktensor()
Y = ttb.ktensor.from_data(X.weights, X.factor_matrices) # Recreate X.
Y

## Creating an empty ktensor

In [ ]:
Z = ttb.ktensor() # Empty ktensor.
Z

## Use full or tensor to convert a ktensor to a tensor

In [ ]:
X = generate_sample_ktensor()
X.full() # Converts to a tensor.

In [ ]:
ttb.tensor.from_tensor_type(X) # Same as above.

## Use double to convert a ktensor to a multidimensional array

In [ ]:
X = generate_sample_ktensor()
X.double() # Converts to an array.

## Use tendiag or sptendiag to convert a ktensor to a ttensor

A ktensor can be regarded as a ttensor with a diagonal core.

### <font color='red'>**TODO:** : Depends on Open Issue</font> (https://github.com/sandialabs/pyttb/issues/148)

In [ ]:
if False:
    X = generate_sample_ktensor()
    R = len(X.weights) # Number of factors in X.
    # core = ttb.tendiag(X.weights, ((R, ) * X.ndims)) # Create a diagonal core.
    # Y = ttb.ttensor.from_data(core, X.factor_matrices)

    """
    --- Simulating correct tendiag() functionality in lieu of line 9 ---
    """
    core = ttb.tensor.from_data(
        np.array([
            [
                [5.0, 0.0],
                [0.0, 0.0]
            ],
            [
                [0.0, 0.0],
                [0.0, 0.25]
            ]
        ])
    )
    Y = ttb.ttensor.from_data(core, X.factor_matrices)
    Y

In [ ]:
(X.full() - Y.full()).norm() # They are the same.

### <font color='red'>**TODO:** : Depends on Open Issue</font> (https://github.com/sandialabs/pyttb/issues/61)
Confirm that it's fine to convert `core` into a tensor-type; bug-> core of type sptensor is disallowed due to `ValueError: TTENSOR doesn't yet support generic cores, only tensor` when trying to instance Y below

In [ ]:
if False: 
    core = ttb.sptendiag(X.weights, ((R, ) * X.ndims)) # Sparse diagonal core.
    """
    --- Converting core to tensor else ValueError for Y-instancing ---
    """
    core = ttb.tensor.from_tensor_type(core) # <--- CONVERSION HERE
    Y = ttb.ttensor.from_data(core, X.factor_matrices) # Assemble the ttensor
    Y

In [ ]:
if False:
    (X.full() - Y.full()).norm() # They are the same.

## Use ndims and size for the dimensions of a ktensor

In [ ]:
X = generate_sample_ktensor()
X.ndims # Number of dimensions.

In [ ]:
X.shape # Tuple of the sizes.

In [ ]:
X.shape[1] # Size of the 2nd mode.

## Subscripted reference for a ktensor

In [ ]:
X = generate_sample_ktensor()
X[0][0][0] # Assemble the (0,0,0) element

In [ ]:
X.weights[1] # Weight of the 2nd factor.

In [ ]:
X.factor_matrices[1] # Extract a matrix.

In [ ]:
X[1] # Same as above.

## Subscripted assignment for a ktensor

In [ ]:
X = generate_sample_ktensor()
X.weights = np.ones(X.weights.size) # Insert new multipliers.
X

### <font color='red'>**TODO:** : Depends on Open Issue</font> (https://github.com/sandialabs/pyttb/issues/165)

In [ ]:
X.weights[0] = 7 # Change a single element of weights.

In [ ]:
X[2][:,[0]] = np.ones((2,1))
X[2][:,[0]]

In [ ]:
X.factor_matrices[2][:, [0]] = np.ones((2,1)) # Change the matrix for mode 3.
X[2][:,[0]]

### <font color='red'>**TODO:** : Depends on Open Issue</font> (https://github.com/sandialabs/pyttb/issues/161)

## Use end for the last array index

In [ ]:
if False:
    X = generate_sample_ktensor()
    X[1:2,X.end(1):,:]

In [ ]:
if False:
    X = generate_sample_ktensor()
    X[0][0][1:X.end()] # Calculates X[0][0][1]

### <font color='red'>**TODO:** : Depends on Open Issue</font> (https://github.com/sandialabs/pyttb/issues/161)

In [ ]:
if False:
    X[X.end()]

## Adding and subtracting ktensors

Adding two ktensors is the same as concatenating the matrices.

In [ ]:
np.random.seed(0)
X = ttb.ktensor.from_factor_matrices([np.random.rand(4,2), np.random.rand(2,2),np.random.rand(3,2)]) # Data.
Y = ttb.ktensor.from_factor_matrices([np.random.rand(4,2), np.random.rand(2,2),np.random.rand(3,2)]) # More data.
X, Y

In [ ]:
Z = X + Y # Concatenates the factor matrices.
Z

In [ ]:
Z = X - Y # Concatenates as with plus, but changes the weights.
Z

In [ ]:
(Z.full() - (X.full() - Y.full())).norm() # Should be zero.

## Basic operations with a ktensor

In [ ]:
+X # Calls uplus.

In [ ]:
-X # Calls uminus.

In [ ]:
5*X # Calls mtimes.

## Use permute to reorder the modes of a ktensor

In [ ]:
np.random.seed(0)
X = ttb.ktensor.from_factor_matrices([np.random.rand(4,2), np.random.rand(2,2),np.random.rand(3,2)]) # Data.
X.permute(np.array((1,2,0))) # Reorders modes of X.

## Use arrange to normalize the factors of a ktensor

The function `arrange` normalizes the columns of the factors and then arranges the rank-one pieces in decreasing order of size.

In [ ]:
np.random.seed(0)
X = ttb.ktensor.from_factor_matrices([np.random.rand(3,2), np.random.rand(4,2),np.random.rand(2,2)]) # Unit weights.
X

In [ ]:
X.arrange() # Normalized and rearranged.
X

## Use fixsigns for sign indeterminacies in a ktensor

The largest magnitude entry for each factor is changed to be positive provided that we can flip the signs of pairs of vectors in that rank-1 component.

In [ ]:
np.random.seed(0)
X = ttb.ktensor.from_factor_matrices([np.random.rand(4,2), np.random.rand(2,2),np.random.rand(3,2)]) # Data.
Y = X
Y.factor_matrices[0][:,0] = -Y.factor_matrices[0][:,0] # switch the sign on a pair of columns
Y.factor_matrices[1][:,0] = -Y.factor_matrices[1][:,0]
Y

In [ ]:
Y.fixsigns()

## Use ktensor to store the 'skinny' SVD of a matrix

### <font color='red'>**TODO:** : Depends on Open Issue</font> (https://github.com/sandialabs/pyttb/issues/167)

In [ ]:
np.random.seed(0)
A = np.random.rand(4,3)
A

In [ ]:
[U,S,V] = np.linalg.svd(A, full_matrices=False) # Compute the SVD.
X = ttb.ktensor.from_data(S, [U, V]) # Store the SVD as a ktensor.
X

In [ ]:
print(f"U*S*V:\n{U@np.diag(S)@V}")
print(f"\nX.factor_matrices[0]@np.diag(X.weights)@X.factor_matrices[1]:\n\
{X.factor_matrices[0]@np.diag(X.weights)@X.factor_matrices[1]}")
print(f"\nX.full():\n{X.full()}") # Reassemble the original matrix.

## Displaying a ktensor

In [ ]:
X = generate_sample_ktensor()
X

In [ ]:
print(X)

## Displaying data

### <font color='red'>**TODO:** : Depends on Open Issue</font> (https://github.com/sandialabs/pyttb/issues/166)

In [ ]:
# not implemented in pyttb